# Finding the Roots of a Polynomial with Eigenvalues and the QR Method

This notebook is based on the project **“Finding the Roots of a Polynomial with Eigenvalues”**.
It shows how to:

- Represent a polynomial as a *companion matrix*.
- Use eigenvalues of the companion matrix to recover the roots of the polynomial.
- Approximate those eigenvalues numerically using the **QR iteration** method.

All the code uses only `numpy` and can be adapted to your own polynomials.

## 1. Polynomials and their roots

A real polynomial of degree $n$ can be written as

$$p(t) = a_n t^n + a_{n-1} t^{n-1} + \cdots + a_1 t + a_0, \qquad a_n \ne 0.$$

A **root** of $p$ is a number $t_*$ such that $p(t_*) = 0$.

For $n = 2$ there is the quadratic formula. For $n = 3$ and $n = 4$ there are more complicated
closed-form formulas. For $n \ge 5$ there is no general formula using only arithmetic operations
and radicals, so in practice we rely on numerical methods.

The idea in this project is: instead of solving $p(t) = 0$ directly, we will build a matrix whose
eigenvalues are exactly the roots of $p$.

## 2. Companion matrices: from polynomials to eigenvalues

Let $p(t)$ be a **monic** polynomial of degree $n$ (leading coefficient 1):

$$p(t) = t^n + a_{n-1} t^{n-1} + \cdots + a_1 t + a_0.$$

The **companion matrix** $C_p$ associated with $p$ is the $n\times n$ matrix

$$
C_p =
\begin{bmatrix}
0 & 0 & \cdots & 0 & -a_0 \\
1 & 0 & \cdots & 0 & -a_1 \\
0 & 1 & \cdots & 0 & -a_2 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & \cdots & 1 & -a_{n-1}
\end{bmatrix}.
$$

One can show that the characteristic polynomial of $C_p$ is

$$\chi_{C_p}(\lambda) = \det(\lambda I - C_p) = p(\lambda).$$

Therefore, the **eigenvalues of the companion matrix are exactly the roots of the polynomial**.

If $p(t) = a_n t^n + \cdots + a_0$ with $a_n \ne 0$, we can divide by $a_n$ to get a monic
polynomial with the same roots. In the code below we will accept general coefficients and
internally rescale to the monic case.

In [1]:
import numpy as np

def companion_matrix(coeffs):
    """Build the companion matrix of a real polynomial.

    Parameters
    ----------
    coeffs : array_like
        Polynomial coefficients [a_n, a_{n-1}, ..., a_0]
        so that p(t) = a_n t^n + ... + a_0, with a_n != 0.

    Returns
    -------
    C : ndarray, shape (n, n)
        Companion matrix whose eigenvalues are the roots of p.
    """
    coeffs = np.asarray(coeffs, dtype=float)
    if coeffs.ndim != 1:
        raise ValueError("coeffs must be a 1D array of coefficients.")
    if coeffs[0] == 0:
        raise ValueError("Leading coefficient must be nonzero.")

    # Make polynomial monic: divide all coefficients by the leading one
    a = coeffs / coeffs[0]
    n = len(a) - 1  # degree
    if n < 1:
        raise ValueError("Degree must be at least 1.")

    C = np.zeros((n, n), dtype=float)
    # Subdiagonal of ones
    if n > 1:
        C[1:, :-1] = np.eye(n - 1)
    # Last column: -a_0, -a_1, ..., -a_{n-1}
    a_tail = a[1:]
    C[:, -1] = -a_tail[::-1]
    return C

In [2]:
# Example: build a companion matrix and compare its eigenvalues with known roots

# Take a polynomial with known roots: (t - 1)(t + 2)(t - 3)
true_roots = np.array([1.0, -2.0, 3.0])
coeffs = np.poly(true_roots)  # returns [1, a2, a1, a0]
print("Polynomial coefficients (highest degree first):")
print(coeffs)

C = companion_matrix(coeffs)
print("\nCompanion matrix C:")
print(C)

eigvals = np.linalg.eigvals(C)
print("\nEigenvalues of C (unsorted):")
print(eigvals)

print("\nSorted eigenvalues vs true roots:")
print(np.sort(eigvals))
print(np.sort(true_roots))

Polynomial coefficients (highest degree first):
[ 1. -2. -5.  6.]

Companion matrix C:
[[ 0.  0. -6.]
 [ 1.  0.  5.]
 [ 0.  1.  2.]]

Eigenvalues of C (unsorted):
[-2.  1.  3.]

Sorted eigenvalues vs true roots:
[-2.  1.  3.]
[-2.  1.  3.]


## 3. The QR method for eigenvalues

The project uses the **QR method** to approximate the eigenvalues of a matrix.

Given a square matrix $A$, one step of the QR iteration is:

1. Compute the QR factorization $A = Q R$, where $Q$ is orthogonal and $R$ is upper triangular.
2. Form the new matrix $A_{\text{new}} = RQ$.

If we repeat this process,

$$A_0 = A, \qquad A_{k+1} = R_k Q_k \quad \text{where } A_k = Q_k R_k,$$

then (under reasonable assumptions) the matrices $A_k$ tend to an upper triangular matrix.
The eigenvalues of $A_k$ are the same as those of $A$ for all $k$, so the diagonal of the
limit matrix gives approximations to the eigenvalues.

In the companion-matrix setting this means: the diagonal entries of $A_k$ approximate the
roots of the polynomial.

In [3]:
def qr_eigenvalues(A, max_iter=1000, tol=1e-10, return_history=False):
    """Approximate eigenvalues of A using the basic QR iteration.

    Parameters
    ----------
    A : (n, n) array_like
        Real square matrix.
    max_iter : int
        Maximum number of QR steps.
    tol : float
        Convergence tolerance for the entries below the diagonal.
    return_history : bool
        If True, also return the sequence of A_k matrices.

    Returns
    -------
    eigs : ndarray, shape (n,)
        Approximate eigenvalues (diagonal of the final A_k).
    Ak : ndarray
        Final A_k matrix.
    k : int
        Number of iterations performed.
    history : list of ndarray (optional)
        If return_history is True.
    """
    A_k = np.array(A, dtype=float)
    if A_k.shape[0] != A_k.shape[1]:
        raise ValueError("A must be square.")

    history = [A_k.copy()] if return_history else None

    for k in range(1, max_iter + 1):
        Q, R = np.linalg.qr(A_k)
        A_k = R @ Q
        if return_history:
            history.append(A_k.copy())
        # Convergence check: norm of strictly lower-triangular part
        off = np.tril(A_k, k=-1)
        err = np.linalg.norm(off, ord='fro')
        if err < tol:
            break

    if return_history:
        return np.diag(A_k), A_k, k, history
    else:
        return np.diag(A_k), A_k, k

In [4]:
# Apply QR iteration to the companion matrix of the example polynomial

Cp = companion_matrix(coeffs)
eig_qr, A_final, iters = qr_eigenvalues(Cp, max_iter=500, tol=1e-12)

print(f"Number of QR iterations: {iters}")
print("Approximate eigenvalues from QR iteration (unsorted):")
print(eig_qr)
print("\nSorted eigenvalues vs true roots:")
print(np.sort(eig_qr))
print(np.sort(true_roots))

def eval_poly(coeffs, t):
    """Evaluate a polynomial with coefficients [a_n,...,a_0] at points t."""
    t = np.asarray(t)
    res = np.zeros_like(t, dtype=float)
    for a in coeffs:
        res = res * t + a
    return res

residuals = eval_poly(coeffs, eig_qr)
print("\nPolynomial evaluated at approximated roots:")
print(residuals)

Number of QR iterations: 73
Approximate eigenvalues from QR iteration (unsorted):
[ 3. -2.  1.]

Sorted eigenvalues vs true roots:
[-2.  1.  3.]
[-2.  1.  3.]

Polynomial evaluated at approximated roots:
[-6.97664149e-12  1.05089271e-11  5.32907052e-15]


## 4. A convenience function: approximate polynomial roots via QR

We now wrap everything into a single helper function that:

1. Builds the companion matrix of a polynomial.
2. Runs QR iteration until the entries below the diagonal are small.
3. Returns the diagonal entries as approximate roots.

In [5]:
def roots_via_qr(coeffs, max_iter=1000, tol=1e-10, verbose=True):
    """Approximate the roots of a real polynomial using the QR method.

    Parameters
    ----------
    coeffs : array_like
        Polynomial coefficients [a_n, ..., a_0].
    max_iter : int
        Maximum number of QR iterations.
    tol : float
        Tolerance for convergence (norm of sub-diagonal entries).
    verbose : bool
        If True, print convergence information.

    Returns
    -------
    roots : ndarray
        Approximate roots (eigenvalues of the companion matrix).
    """
    C = companion_matrix(coeffs)
    eigs, A_final, iters = qr_eigenvalues(C, max_iter=max_iter, tol=tol)
    if verbose:
        off = np.tril(A_final, k=-1)
        err = np.linalg.norm(off, ord='fro')
        print(f"QR iterations: {iters}, sub-diagonal Frobenius norm = {err:.3e}")
    return eigs

In [6]:
# Try the helper function on a few examples

# Example 1: cubic with roots 1, -2, 3
roots1_true = np.array([1.0, -2.0, 3.0])
coeffs1 = np.poly(roots1_true)
roots1_qr = roots_via_qr(coeffs1, max_iter=500, tol=1e-12)
print("Example 1:")
print("True roots (sorted):    ", np.sort(roots1_true))
print("QR-approximated roots:  ", np.sort(roots1_qr))

# Example 2: quadratic p(t) = t^2 + 2t - 4 (roots -1 ± sqrt(5))
coeffs2 = np.array([1.0, 2.0, -4.0])  # t^2 + 2 t - 4
roots2_qr = roots_via_qr(coeffs2, max_iter=100, tol=1e-14)
print("\nExample 2:")
print("QR-approximated roots:", np.sort(roots2_qr))
print("numpy.roots:           ", np.sort(np.roots(coeffs2)))

QR iterations: 73, sub-diagonal Frobenius norm = 6.724e-13
Example 1:
True roots (sorted):     [-2.  1.  3.]
QR-approximated roots:   [-2.  1.  3.]
QR iterations: 36, sub-diagonal Frobenius norm = 7.098e-15

Example 2:
QR-approximated roots: [-3.23606798  1.23606798]
numpy.roots:            [-3.23606798  1.23606798]


## 5. Notes and limitations

- The basic QR iteration implemented here is the same conceptual algorithm described in the project,
  but practical eigenvalue solvers use extra techniques (like *shifts* and *deflation*) to make
  convergence faster and more robust.
- For polynomials with complex roots, the real QR iteration on the companion matrix may produce
  $2\times2$ blocks rather than individual diagonal entries; more sophisticated handling is then
  needed to extract complex conjugate eigenvalues accurately.
- Libraries such as `numpy.roots` and `numpy.linalg.eigvals` already use highly optimized versions of
  these ideas. The point of this notebook is to connect the theory with an explicit, readable
  implementation.